In [1]:
import cv2
import numpy as np
import pytesseract
import grip
import time
from dataclasses import dataclass

cap = cv2.VideoCapture("/Users/akshar/Downloads/Humans in Autonomy/DroneFootageAnalysis/SampleVid.mp4")
REFERENCE_INSPECTION_FRAME = cv2.imread("/Users/akshar/Downloads/Humans in Autonomy/DroneFootageAnalysis/blankFrame.jpg")
CROPPED_REFERENCE = cv2.imread("/Users/akshar/Downloads/Humans in Autonomy/DroneFootageAnalysis/ReferenceUnderJoystick.jpg")
LEFT_JOYSTICK = cv2.imread("/Users/akshar/Downloads/Humans in Autonomy/DroneFootageAnalysis/leftJoystick.jpg")
RIGHT_JOYSTICK = cv2.imread("/Users/akshar/Downloads/Humans in Autonomy/DroneFootageAnalysis/rightJoystick.jpg")
TESSERACT_PATH = "/Users/akshar/Downloads/Humans in Autonomy/DroneFootageAnalysis/tesseractPattern.txt"

LEFT_JOYSTICK_DIFFERENCE_THRESHOLD = 300000
RIGHT_JOYSTICK_DIFFERENCE_THRESHOLD = 250000
INSPECTION_MODE_THRESHOLD = 400000

start_frame_number = 5500
cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame_number)
cv2.startWindowThread()

clicks = []
times = []
ocr_error = []

@dataclass
class ClickEvent:
    side: float
    time: float

def isInspectionMode(left, right):
    left_diff = cv2.subtract(left, LEFT_JOYSTICK)
    right_diff = cv2.subtract(right, RIGHT_JOYSTICK)
    
    if (left_diff.sum() + right_diff.sum())/2 < INSPECTION_MODE_THRESHOLD:
        return True
    else:
        return False
        
def isLeftClick(frame, reference):
    difference = cv2.subtract(frame, reference)
    print ("l ", difference.sum())
    if difference.sum() > LEFT_JOYSTICK_DIFFERENCE_THRESHOLD:
        print ("returning left click")
        cv2.imshow("left difference", difference)
        return True
    else:
        return False
    
def isRightClick(frame, reference):
    difference = cv2.subtract(frame, reference)
    print ("r ", difference.sum())
    if difference.sum() > RIGHT_JOYSTICK_DIFFERENCE_THRESHOLD:
        print ("returning right click")
        cv2.imshow("right difference", difference)
        return True
    else:
        return False

def flightTime(frame):
    time_frame = frame[10:75, 600:700]
    cv2.imshow("time", time_frame)
    time_str = pytesseract.image_to_string(time_frame, config='--psm 6')
    try:
        seconds = (time.strptime(time_str, '%M:%S')[4]*60) + time.strptime(time_str, '%M:%S')[5]
        return seconds
    except:
        print ("BAD OCR. READ {}".format(time_str))
        if time_str not in ocr_error:
            seconds = times[-1]+1
            ocr_error.append(time_str)
        else:
            seconds = times[-1]
        print ("returning %d" % seconds)
        return seconds
    
def checkJoysticks(frame):
    #frame - cv2.medianBlur(frame, 5)
    #LEFT_JOYSTICK = cv2.medianBlur(LEFT_JOYSTICK, 5)
    #RIGHT_JOYSTICK = cv2.medianBlur(RIGHT_JOYSTICK, 5)
    leftJoystick = frame[160:420, 20:280]
    rightJoystick = frame[160:420:, 1000:1260]
    time = flightTime(frame)
    if isInspectionMode(leftJoystick, rightJoystick):
        if isLeftClick(leftJoystick, LEFT_JOYSTICK) and not isRightClick(rightJoystick, RIGHT_JOYSTICK):            
            if time not in times or not time:
                clicks.append(ClickEvent(1, time))
                times.append(time)
        if isRightClick(rightJoystick, RIGHT_JOYSTICK):
            if time not in times or not time:
                clicks.append(ClickEvent(2, time))
                times.append(time)
    

In [2]:
print ("running...")
print (cap.isOpened())
while cap.isOpened():
    ret, frame = cap.read()
    if ret == True:
        cv2.imshow("frame", frame)
        checkJoysticks(frame)
        print(clicks)
        print(times)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

running...
True
l  311550
returning left click
r  224028
r  224028
[ClickEvent(side=1, time=136)]
[136]
l  304696
returning left click
r  224058
r  224058
[ClickEvent(side=1, time=136)]
[136]
l  295239
r  224058
[ClickEvent(side=1, time=136)]
[136]
l  295243
r  224058
[ClickEvent(side=1, time=136)]
[136]
l  295243
r  224058
[ClickEvent(side=1, time=136)]
[136]
l  295243
r  224058
[ClickEvent(side=1, time=136)]
[136]
l  295243
r  224058
[ClickEvent(side=1, time=136)]
[136]
l  295243
r  224058
[ClickEvent(side=1, time=136)]
[136]
l  295243
r  224058
[ClickEvent(side=1, time=136)]
[136]
l  295243
r  224058
[ClickEvent(side=1, time=136)]
[136]
l  295243
r  224058
[ClickEvent(side=1, time=136)]
[136]
l  295243
r  224058
[ClickEvent(side=1, time=136)]
[136]
l  295243
r  224058
[ClickEvent(side=1, time=136)]
[136]
l  295243
r  224058
[ClickEvent(side=1, time=136)]
[136]
l  295243
r  224058
[ClickEvent(side=1, time=136)]
[136]
l  295243
r  224058
[ClickEvent(side=1, time=136)]
[136]
l  295243


l  476855
returning left click
r  244730
r  244730
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139)]
[136, 138, 139]
l  476855
returning left click
r  244730
r  244730
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139)]
[136, 138, 139]
l  476855
returning left click
r  244730
r  244730
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139)]
[136, 138, 139]
l  468689
returning left click
r  244730
r  244730
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139)]
[136, 138, 139]
l  372435
returning left click
r  224302
r  224302
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139)]
[136, 138, 139]
l  353698
returning left click
r  224321
r  224321
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139)]
[136, 138, 139]
l  344524
returning left click
r  224321
r  224321
[

BAD OCR. READ 7A
returning 140
l  295670
r  224058
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139), ClickEvent(side=1, time=140)]
[136, 138, 139, 140]
BAD OCR. READ 7A
returning 140
l  295670
r  224058
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139), ClickEvent(side=1, time=140)]
[136, 138, 139, 140]
BAD OCR. READ 7A
returning 140
l  295670
r  224058
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139), ClickEvent(side=1, time=140)]
[136, 138, 139, 140]
BAD OCR. READ 7A
returning 140
l  295670
r  224058
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139), ClickEvent(side=1, time=140)]
[136, 138, 139, 140]
BAD OCR. READ 7A
returning 140
l  295670
r  224058
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139), ClickEvent(side=1, time=140)]
[136, 138, 139, 140]
BAD OCR. READ 7A
returning 140
l  3

l  295383
r  224001
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139), ClickEvent(side=1, time=140), ClickEvent(side=1, time=142)]
[136, 138, 139, 140, 142]
l  295383
r  224001
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139), ClickEvent(side=1, time=140), ClickEvent(side=1, time=142)]
[136, 138, 139, 140, 142]
l  295383
r  224001
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139), ClickEvent(side=1, time=140), ClickEvent(side=1, time=142)]
[136, 138, 139, 140, 142]
l  295383
r  224001
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139), ClickEvent(side=1, time=140), ClickEvent(side=1, time=142)]
[136, 138, 139, 140, 142]
l  295383
r  224001
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139), ClickEvent(side=1, time=140), ClickEvent(side=1, time=142)]
[136, 138, 139, 140, 142]
BAD OCR. READ W

BAD OCR. READ Ay.)
returning 143
l  393744
returning left click
r  222652
r  222652
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139), ClickEvent(side=1, time=140), ClickEvent(side=1, time=142), ClickEvent(side=1, time=143)]
[136, 138, 139, 140, 142, 143]
BAD OCR. READ Ay.)
returning 143
l  393744
returning left click
r  222652
r  222652
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139), ClickEvent(side=1, time=140), ClickEvent(side=1, time=142), ClickEvent(side=1, time=143)]
[136, 138, 139, 140, 142, 143]
BAD OCR. READ Ay.)
returning 143
l  393766
returning left click
r  222652
r  222652
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139), ClickEvent(side=1, time=140), ClickEvent(side=1, time=142), ClickEvent(side=1, time=143)]
[136, 138, 139, 140, 142, 143]
BAD OCR. READ Ay.)
returning 143
l  393728
returning left click
r  222652
r  222652
[ClickEvent(side=1, time=136

...

l  294839
r  223814
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139), ClickEvent(side=1, time=140), ClickEvent(side=1, time=142), ClickEvent(side=1, time=143), ClickEvent(side=1, time=146), ClickEvent(side=1, time=147), ClickEvent(side=1, time=148), ClickEvent(side=1, time=149), ClickEvent(side=2, time=151), ClickEvent(side=2, time=154), ClickEvent(side=2, time=156), ClickEvent(side=2, time=157), ClickEvent(side=2, time=165)]
[136, 138, 139, 140, 142, 143, 146, 147, 148, 149, 151, 154, 156, 157, 165]
l  294839
r  223814
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139), ClickEvent(side=1, time=140), ClickEvent(side=1, time=142), ClickEvent(side=1, time=143), ClickEvent(side=1, time=146), ClickEvent(side=1, time=147), ClickEvent(side=1, time=148), ClickEvent(side=1, time=149), ClickEvent(side=2, time=151), ClickEvent(side=2, time=154), ClickEvent(side=2, time=156), ClickEvent(side=2, time=157), ClickEvent(s

l  294806
r  223824
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139), ClickEvent(side=1, time=140), ClickEvent(side=1, time=142), ClickEvent(side=1, time=143), ClickEvent(side=1, time=146), ClickEvent(side=1, time=147), ClickEvent(side=1, time=148), ClickEvent(side=1, time=149), ClickEvent(side=2, time=151), ClickEvent(side=2, time=154), ClickEvent(side=2, time=156), ClickEvent(side=2, time=157), ClickEvent(side=2, time=165)]
[136, 138, 139, 140, 142, 143, 146, 147, 148, 149, 151, 154, 156, 157, 165]
l  294806
r  223824
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139), ClickEvent(side=1, time=140), ClickEvent(side=1, time=142), ClickEvent(side=1, time=143), ClickEvent(side=1, time=146), ClickEvent(side=1, time=147), ClickEvent(side=1, time=148), ClickEvent(side=1, time=149), ClickEvent(side=2, time=151), ClickEvent(side=2, time=154), ClickEvent(side=2, time=156), ClickEvent(side=2, time=157), ClickEvent(s

l  294806
r  223824
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139), ClickEvent(side=1, time=140), ClickEvent(side=1, time=142), ClickEvent(side=1, time=143), ClickEvent(side=1, time=146), ClickEvent(side=1, time=147), ClickEvent(side=1, time=148), ClickEvent(side=1, time=149), ClickEvent(side=2, time=151), ClickEvent(side=2, time=154), ClickEvent(side=2, time=156), ClickEvent(side=2, time=157), ClickEvent(side=2, time=165)]
[136, 138, 139, 140, 142, 143, 146, 147, 148, 149, 151, 154, 156, 157, 165]
l  294806
r  223824
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139), ClickEvent(side=1, time=140), ClickEvent(side=1, time=142), ClickEvent(side=1, time=143), ClickEvent(side=1, time=146), ClickEvent(side=1, time=147), ClickEvent(side=1, time=148), ClickEvent(side=1, time=149), ClickEvent(side=2, time=151), ClickEvent(side=2, time=154), ClickEvent(side=2, time=156), ClickEvent(side=2, time=157), ClickEvent(s

l  294806
r  223842
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139), ClickEvent(side=1, time=140), ClickEvent(side=1, time=142), ClickEvent(side=1, time=143), ClickEvent(side=1, time=146), ClickEvent(side=1, time=147), ClickEvent(side=1, time=148), ClickEvent(side=1, time=149), ClickEvent(side=2, time=151), ClickEvent(side=2, time=154), ClickEvent(side=2, time=156), ClickEvent(side=2, time=157), ClickEvent(side=2, time=165)]
[136, 138, 139, 140, 142, 143, 146, 147, 148, 149, 151, 154, 156, 157, 165]
l  294806
r  223842
[ClickEvent(side=1, time=136), ClickEvent(side=1, time=138), ClickEvent(side=1, time=139), ClickEvent(side=1, time=140), ClickEvent(side=1, time=142), ClickEvent(side=1, time=143), ClickEvent(side=1, time=146), ClickEvent(side=1, time=147), ClickEvent(side=1, time=148), ClickEvent(side=1, time=149), ClickEvent(side=2, time=151), ClickEvent(side=2, time=154), ClickEvent(side=2, time=156), ClickEvent(side=2, time=157), ClickEvent(s

KeyboardInterrupt: 